# 3. Spatial Interaction Models


In [3]:
# import all the necessary libraries
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.special import expit  
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import scipy.stats
from math import sqrt
import statsmodels.formula.api as smf
from scipy.stats import norm

## 3. models:

| Models             | Formula                                                                                     | Definition                                                                                                                                                                                    |
|--------------------|---------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Gravity            | $$ T_{ij} = kO_i^\alpha D_j^\gamma d_{ij}^{-\beta} $$                                       | Predicts interaction flow from origin i to destination j , based on 'mass' of origin and destination, and the distance/cost between them.                                          |
| Unconstrained      | $$ T_{ij} = kO_i^\alpha D_j^\gamma d_{ij}^{-\beta} $$                                      | Similar to the Gravity Model without any additional constraints, this framework allows for interactions or flows between spatial units without limitations.                                    |
| Singly Constrained | Origin-Constrained: $$ T_{ij} = A_i O_i D_j d_{ij}^{-\beta} $$ Destination-Constrained: $$ T_{ij} = O_i B_j D_j d_{ij}^{-\beta} $$ | Includes a constraint on either the supply side (origin) or demand side (destination). The flow from each origin or to each destination sums to a known total. Balancing factors $$ A_i $$ or $$ B_j $$ are introduced. |
| Doubly Constrained | $$ T_{ij} = A_i B_j O_i D_j d_{ij}^{-\beta} $$   $$A_i = \frac{1}{\sum\limits_{j} B_j D_j d_{ij}^{-\beta}} $$ $$\quad B_j = \frac{1}{\sum\limits_{i} A_i O_i d_{ij}^{-\beta}}$$        | Extends the Singly Constrained model by incorporating both origin and destination constraints. It ensures the total outflow from each origin and the total inflow to each destination match known totals.                 |


## 3.2 Model Parameter Calibration
use single constrained model.

In [2]:
#set up the metric calculations
def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

In [5]:
london_flow_df = pd.read_csv('london_flows.csv')

clear 0s

In [6]:
for column in ['population', 'jobs', 'distance']:
    london_flow_df = london_flow_df[london_flow_df[column] != 0]

london_flow_df = london_flow_df.reset_index()

normalise the columns:

In [8]:
london_flow_df['population_log'] = london_flow_df['population'].apply(lambda x: np.log(x))
london_flow_df['jobs_log'] = london_flow_df['jobs'].apply(lambda x: np.log(x))
london_flow_df['distance_log'] = london_flow_df['distance'].apply(lambda x: np.log(x))

In [9]:
london_flow_df.to_csv('london_flows_processed.csv', index=False)

singly constrained model: (dest constrained)

In [12]:
sc_df = pd.read_csv('london_flows_processed.csv')

In [13]:
sc_df.head()

,station_origin,station_destination,flows,population,jobs,distance,population_log,jobs_log,distance_log
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395262,11.271478,9.003504
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274


In [26]:
sc_dest_c_model = smf.glm(formula = 'flows ~ station_destination + jobs_log + distance_log-1', 
                    data=sc_df, family=sm.families.Poisson()).fit()

In [23]:
# store sc_model.summary() to pd.dataframe:
summary_001 = sc_dest_c_model.summary()

summary_001 = summary_001.tables[1]

summary_001 = pd.DataFrame(data=summary_001.data[1:], columns=summary_001.data[0])

summary_001 = summary_001.set_index(summary_001.columns[0])


#.to_csv('sc_model_summary_001.csv')

In [24]:
summary_001.to_csv('sc_model_summary_001.csv')

In [25]:
print(sc_dest_c_model())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61413
Model:                            GLM   Df Residuals:                    61014
Model Family:                 Poisson   Df Model:                          398
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.7321e+06
Date:                Sun, 28 Apr 2024   Deviance:                   3.2919e+06
Time:                        20:20:54   Pearson chi2:                 7.04e+06
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------

In [30]:
predictions = sc_dest_c_model.get_prediction(sc_df[["station_destination", "jobs_log", "distance_log"]])
predictions_summary_frame = predictions.summary_frame()
sc_df["attrsimFitted"] = round(predictions_summary_frame["mean"],0)
#now we can create pivot table to turn paired list into matrix (and compute the margins as well)
sc_df_2 = sc_df.pivot_table(values ="attrsimFitted", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
sc_df_2

/tmp/ipykernel_124122/2851209573.py:5: FutureWarning: The provided callable <function sum at 0x7fcc6430edd0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  sc_df_2 = sc_df.pivot_table(values ="attrsimFitted", index="station_origin", columns = "station_destination",
/tmp/ipykernel_124122/2851209573.py:5: FutureWarning: The provided callable <function sum at 0x7fcc6430edd0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  sc_df_2 = sc_df.pivot_table(values ="attrsimFitted", index="station_origin", columns = "station_destination",
/tmp/ipykernel_124122/2851209573.py:5: FutureWarning: The provided callable <function sum at 0x7fcc6430edd0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be u

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.0,2503.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1258.0
Acton Town,NaN,NaN,NaN,20.0,19.0,NaN,10.0,2.0,NaN,20.0,...,29.0,4.0,6.0,11.0,NaN,2.0,NaN,3.0,NaN,4039.0
Aldgate,NaN,NaN,7.0,NaN,115.0,NaN,NaN,1.0,NaN,59.0,...,25.0,NaN,10.0,7.0,NaN,4.0,NaN,4.0,NaN,8886.0
Aldgate East,NaN,NaN,7.0,119.0,NaN,NaN,3.0,1.0,NaN,56.0,...,25.0,3.0,10.0,7.0,NaN,4.0,NaN,4.0,NaN,8657.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,5.0,21.0,22.0,NaN,NaN,NaN,NaN,18.0,...,17.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,3121.0
Woodgrange Park,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566.0
Woodside Park,NaN,NaN,5.0,20.0,19.0,NaN,3.0,NaN,NaN,21.0,...,19.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,3085.0


In [31]:
def decay_function(distance, b):
    return np.exp(-b * distance)

# We'll use an optimization routine to find the best-fitting 'b'
def calibrate_b(df):
    # Define the objective function that we want to minimize (negative log-likelihood)
    def objective(b):
        # Calculate the expected flows using the decay function
        expected_flows = df['population'] * df['jobs'] * decay_function(df['distance'], b)
        # Avoid dividing by zero
        expected_flows[expected_flows == 0] = 1e-5
        # Calculate the log-likelihood
        log_likelihood = -np.sum(df['flows'] * np.log(expected_flows) - expected_flows)
        return log_likelihood

    # Initial guess
    b0 = 0.1
    # Perform the minimization
    result = minimize(objective, b0, method='L-BFGS-B', bounds=[(0.001, None)])
    return result.x[0]

In [32]:
# Calibrate the 'b' parameter
b_calibrated = calibrate_b(sc_df)
print(f"Calibrated b parameter: {b_calibrated}")

# Now let's add the estimated flows to the dataframe using the calibrated b
sc_df['estimated_flows'] = sc_df['population'] * sc_df['jobs'] * decay_function(sc_df['distance'], b_calibrated)

# Select only the relevant columns for a clear output
output_columns = ['station_origin', 'station_destination', 'flows', 'estimated_flows']
result_df = sc_df[output_columns]

Calibrated b parameter: 0.09997902916113967


In [ ]:
#set out all the fomrulas
formula1 = "Total ~ np.log(Oi1_origpop) + np.log(Dj2_destsal) + np.log(Dist) -1"
formula2 = "Total ~ OrigCodeNew + np.log(Dj2_destsal) + np.log(Dist) -1"
formula3 = "Total ~ np.log(Oi1_origpop) + DestCodeNew + np.log(Dist) -1"
formula4 = "Total ~ OrigCodeNew + DestCodeNew + np.log(Dist) -1"
formula5 = "Total ~ np.log(Oi1_origpop) + np.log(Dj2_destsal) + Dist -1"
formula6 = "Total ~ OrigCodeNew + np.log(Dj2_destsal) + Dist -1"
formula7 = "Total ~ np.log(Oi1_origpop) + DestCodeNew + Dist -1"
formula8 = "Total ~ OrigCodeNew + DestCodeNew + Dist -1"

#create a list of all the formulas
formulas = [formula1, formula2, formula3, formula4, 
            formula5, formula6, formula7, formula8]

#list the models name
models = ["uncosim_pow", "prodsim_pow", "attrsim_pow", "doublesim_pow",
         "uncosim_exp", "prodsim_exp", "attrsim_exp", "doublesim_exp"]

#create a set of tuples to store whether a paramater
#will be in the model or not
model_params = [(True, True, True),
               (False, True, True),
               (True, False, True),
               (False, False, True),
               (True, True, True),
               (False, True, True),
               (True, False, True),
               (False, False, True)]

#create an ampty dictionary to store the results
results = {"Model":models,
          "R2": [],
          "RMSE": [],
          "Alpha":[],
          "Gamma":[],
          "Beta":[]}

#loop over each formula
for i, formula in enumerate(formulas):
    
    #run the specified model
    sim = smf.glm(formula = formula, 
                 data = cdata,
                 family = sm.families.Poisson()).fit()
    #clauclate the estimates
    cdata[models[i]] = sim.mu
    
    #if the alpha paramater is true
    if model_params[i][0] == True:
        
        #if there are three params then it will be 
        #in the third position
        if sum(model_params[i]) == 3:
            
            results["Alpha"].append(sim.params[-3])
            
        #otherwise it will be in the second paramater
        else:
            results["Alpha"].append(sim.params[-2])
            
    #if not then just append nan
    else:
        results["Alpha"].append(np.nan) 
    
    #if the gamma paramater is True
    #then it will always be the second paramater
    if model_params[i][1] == True:
        results["Gamma"].append(sim.params[-2])
    #otherwise add nan
    else:
        results["Gamma"].append(np.nan)
    
    #add the beta to the results (always will be there)
    results["Beta"].append(sim.params[-1])
        
    #add the metrics to the results dictionary
    results["R2"].append(CalcRSqaured(cdata["Total"],cdata[models[i]]))
    results["RMSE"].append(CalcRMSE(cdata["Total"],cdata[models[i]]))

#create a dataframe from the results
results = pd.DataFrame(results)
#print the results
results

## 4. Scenarios


### 4.1 Scenario A:

### 4.2 Scenario B

### 4.3 Discussion